In [24]:
import pandas as pd
import os
import emoji
import regex as re
pd.set_option('display.max_colwidth', None)

In [25]:
folder_path = "../dataset/clockworks-scraper"

In [26]:
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

In [27]:
dfs = [pd.read_csv(os.path.join(folder_path, file), encoding='utf-8') for file in csv_files]
combined_df = pd.concat(dfs, ignore_index=True)

In [29]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23043 entries, 0 to 23042
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   avatarThumbnail    23039 non-null  object 
 1   cid                23042 non-null  float64
 2   createTime         23042 non-null  float64
 3   createTimeISO      23042 non-null  object 
 4   diggCount          23042 non-null  float64
 5   repliesToId        4430 non-null   float64
 6   replyCommentTotal  18612 non-null  float64
 7   submittedVideoUrl  23042 non-null  object 
 8   text               23042 non-null  object 
 9   uid                23042 non-null  float64
 10  uniqueId           23039 non-null  object 
 11  videoWebUrl        23042 non-null  object 
 12  error              1 non-null      object 
 13  input              13744 non-null  object 
 14  likedByAuthor      13744 non-null  object 
 15  pinnedByAuthor     11235 non-null  object 
 16  url                1 n

In [30]:
combined_df.sample(10)

,avatarThumbnail,cid,createTime,createTimeISO,diggCount,repliesToId,replyCommentTotal,submittedVideoUrl,text,uid,uniqueId,videoWebUrl,error,input,likedByAuthor,pinnedByAuthor,url
21454,https://p16-sign-sg.tiktokcdn.com/tos-alisg-avt-0068/eb433a89ad66d5c8ee1a17e640ff2e21~tplv-tiktokx-cropcenter:100:100.jpg?dr=14579&refresh_token=cea03130&x-expires=1742547600&x-signature=ctJ47amLVHGbNFyctc4eZS9BGq8%3D&t=4d5b0474&ps=13740610&shp=30310797&shcp=ff37627b&idc=maliva,7.453402e+18,1.735380e+09,2024-12-28T10:06:46.000Z,0.0,NaN,0.0,https://www.tiktok.com/@serambiherbal/video/7445514816568954119,semuanya di gw...,7.238905e+18,cesita.cristyy,https://www.tiktok.com/@serambiherbal/video/7445514816568954119,NaN,https://www.tiktok.com/@serambiherbal/video/7445514816568954119,False,False,NaN
20114,https://p16-common-sign-va.tiktokcdn-us.com/tos-maliva-avt-0068/84585362c104bea7a65605e663256576~tplv-tiktokx-cropcenter:100:100.jpg?dr=9640&refresh_token=0185706c&x-expires=1742547600&x-signature=CUpy1x%2BFxrKhPuGMGuz%2FTKj6KXE%3D&t=4d5b0474&ps=13740610&shp=30310797&shcp=ff37627b&idc=useast5,7.463355e+18,1.737698e+09,2025-01-24T05:47:39.000Z,0.0,7.373163e+18,NaN,https://www.tiktok.com/@berbagirasa9/video/7288356653718605062,"alhamdulilah masi bernapas smpai skrng.yg penting klo asam lambung kumat pikiran harus enjoy,klo saya langsung kunyah permen karet.",7.018073e+18,user31506351063195,https://www.tiktok.com/@berbagirasa9/video/7288356653718605062,NaN,https://www.tiktok.com/@berbagirasa9/video/7288356653718605062,False,NaN,NaN
15066,https://p16-sign-sg.tiktokcdn.com/tos-alisg-avt-0068/b6e66b979bc1bf8bab4146f5365bc16d~tplv-tiktokx-cropcenter:100:100.jpg?dr=10399&refresh_token=dc70cfb2&x-expires=1742544000&x-signature=4Xt39HXW0E%2BBJZD9KX9y%2BJnL%2F68%3D&t=4d5b0474&ps=13740610&shp=30310797&shcp=ff37627b&idc=no1a,7.409667e+18,1.725198e+09,2024-09-01T13:33:13.000Z,0.0,NaN,0.0,https://www.tiktok.com/@katapsikologi/video/7394700752683355398,@Kamals Xfriends bener2 yang aku rasain sekarang,6.785897e+18,taurus_csl,https://www.tiktok.com/@katapsikologi/video/7394700752683355398,NaN,https://www.tiktok.com/@katapsikologi/video/7394700752683355398,False,False,NaN
8243,https://p16-sign-sg.tiktokcdn.com/tos-alisg-avt-0068/056d611d2bbb70da7748d6dc72d68ed1~tplv-tiktokx-cropcenter:100:100.jpg?dr=10399&refresh_token=37121ad6&x-expires=1741284000&x-signature=RT6t9sz%2Bs3zUuhYP8IhLc7MMB9U%3D&t=4d5b0474&ps=13740610&shp=30310797&shcp=ff37627b&idc=no1a,7.469145e+18,1.739046e+09,2025-02-08T20:19:37.000Z,0.0,NaN,0.0,https://www.tiktok.com/@berbagirasa9/video/7288356653718605062,Berawal dari overthinking😩,6.887084e+18,ayamrechess,https://www.tiktok.com/@berbagirasa9/video/7288356653718605062,NaN,NaN,NaN,NaN,NaN
22706,https://p16-sign-useast2a.tiktokcdn.com/tos-useast2a-avt-0068-giso/f30ba1673713b1419fbe4c3163bb5162~tplv-tiktokx-cropcenter:100:100.jpg?dr=9640&refresh_token=dd2af99e&x-expires=1742547600&x-signature=f3Y3cxbI5gN%2BLHxBd%2BBour6RLck%3D&t=4d5b0474&ps=13740610&shp=30310797&shcp=ff37627b&idc=useast5,7.246605e+18,1.687232e+09,2023-06-20T03:27:14.000Z,0.0,NaN,0.0,https://www.tiktok.com/@tatanasmit/video/7223340763915046171,lagi ngalamin ini😌,6.822896e+18,cirengisiayamsuwir,https://www.tiktok.com/@tatanasmit/video/7223340763915046171,NaN,https://www.tiktok.com/@tatanasmit/video/7223340763915046171,False,False,NaN
1247,https://p16-sign-sg.tiktokcdn.com/tos-alisg-avt-0068/4c17404c52108c29d3aec2ce22be111b~tplv-tiktokx-cropcenter:100:100.jpg?dr=10399&nonce=69224&refresh_token=a0b9897391dfe62246c4852e83c8f529&x-expires=1739779200&x-signature=YcSQhnpMgPymo4SUBxHSFRxzGl8%3D&idc=no1a&ps=13740610&shcp=ff37627b&shp=30310797&t=4d5b0474,7.329505e+18,1.706533e+09,2024-01-29T13:04:01.000Z,1.0,NaN,0.0,https://www.tiktok.com/@sundarindah/video/7329123581722447110,@fzhan__,6.749851e+18,nindyndm,https://www.tiktok.com/@sundarindah/video/7329123581722447110,NaN,NaN,NaN,NaN,NaN
10598,https://p16-common-sign-va.tiktokcdn-us.com/tos-maliva-avt-0068/1cb9ac775c1bfb4d1d95df9f85fa0047~tplv-tikt

In [31]:
# Remove NaN values before processing
data = combined_df.dropna(subset=['text'])

# Text Normalization - Convert to lowercase
data['text'] = data['text'].astype(str).str.lower()

# Removing Mentions, Emojis, and Emoticons
def remove_mentions(text):
    return re.sub(r'@\w+', '', text)

def remove_emojis(text):
    return emoji.replace_emoji(text, replace='')

def remove_emoticons(text):
    emoticon_pattern = r'[:;=8xX][-^]?[\)DPOp\*\(\|\\\/ ]'
    return re.sub(emoticon_pattern, '', text)

def majority_special_chars(text):
    special_chars = re.sub(r'[a-zA-Z0-9\s]', '', text)  # Extract special characters
    return len(special_chars) > len(text) / 2  # Check if majority are special chars

def clean_tiktok_text(text):
    if pd.isna(text):  # Handle missing values
        return ""
    text = remove_mentions(text)  # Remove mentions
    text = remove_emojis(text)    # Remove emojis
    text = remove_emoticons(text) # Remove ASCII emoticons
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

# Apply cleaning function
data['text'] = data['text'].apply(clean_tiktok_text)

# Remove rows where 'text' is empty after cleaning
data = data[data['text'].str.strip() != ""]

# Remove rows where 'text' has 4 characters or less
data = data[data['text'].str.len() > 4]

# Remove rows where the majority of characters are special characters
data = data[~data['text'].apply(majority_special_chars)]

# Remove duplicate text entries
data = data.drop_duplicates(subset=['text'])

# Keep only required columns
data = data[['cid', 'submittedVideoUrl', 'text']]

# Display cleaned text
data.head()

C:\Users\Mufid Musyaffa\AppData\Local\Temp\ipykernel_12960\3432758757.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].astype(str).str.lower()
C:\Users\Mufid Musyaffa\AppData\Local\Temp\ipykernel_12960\3432758757.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(clean_tiktok_text)


,cid,submittedVideoUrl,text
0,7.424290e+18,https://www.tiktok.com/@sundarindah/video/7424130682529516805,sayangnya orang-orang terdekat yang justru membuat mental kita sakit dan lelah
1,7.424297e+18,https://www.tiktok.com/@sundarindah/video/7424130682529516805,"justru malah org trdekat yg menghancurkan mental ku, aku tdk prcya siapapun, caraku mengatasi ini semua dgn meditasi agar lbih tenang"
2,7.467563e+18,https://www.tiktok.com/@sundarindah/video/7424130682529516805,halo ka indah sundari
3,7.424316e+18,https://www.tiktok.com/@sundarindah/video/7424130682529516805,justru orang terdekat malah jadi penyebabnya
4,7.455553e+18,https://www.tiktok.com/@sundarindah/video/7424130682529516805,ka tolong aku


In [32]:
cleaned_df = data

In [33]:
cleaned_df.duplicated(subset=['text']).sum()

0

In [34]:
output_filename = "cleaned_dataset.csv"
cleaned_df.to_csv(output_filename, index=False, encoding='utf-8')

In [35]:
print(f"Dataset bersih telah disimpan di {output_filename}")
cleaned_df.head()

Dataset bersih telah disimpan di cleaned_dataset.csv


,cid,submittedVideoUrl,text
0,7.424290e+18,https://www.tiktok.com/@sundarindah/video/7424130682529516805,sayangnya orang-orang terdekat yang justru membuat mental kita sakit dan lelah
1,7.424297e+18,https://www.tiktok.com/@sundarindah/video/7424130682529516805,"justru malah org trdekat yg menghancurkan mental ku, aku tdk prcya siapapun, caraku mengatasi ini semua dgn meditasi agar lbih tenang"
2,7.467563e+18,https://www.tiktok.com/@sundarindah/video/7424130682529516805,halo ka indah sundari
3,7.424316e+18,https://www.tiktok.com/@sundarindah/video/7424130682529516805,justru orang terdekat malah jadi penyebabnya
4,7.455553e+18,https://www.tiktok.com/@sundarindah/video/7424130682529516805,ka tolong aku


In [36]:
cleaned_df.info()
print(f"Dataset bersih telah disimpan di {output_filename}")
print(f"Jumlah data sebelum: {combined_df.shape[0]}, setelah: {cleaned_df.shape[0]}")

<class 'pandas.core.frame.DataFrame'>
Index: 14021 entries, 0 to 23041
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   cid                14021 non-null  float64
 1   submittedVideoUrl  14021 non-null  object 
 2   text               14021 non-null  object 
dtypes: float64(1), object(2)
memory usage: 438.2+ KB
Dataset bersih telah disimpan di cleaned_dataset.csv
Jumlah data sebelum: 23043, setelah: 14021
